<a href="https://colab.research.google.com/github/ashokmaster83/SummerAnalytics1/blob/main/SAweek2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
import warnings
import random
warnings.filterwarnings('ignore')

# Load data
train_df = pd.read_csv("/content/hacktrain.csv")
test_df = pd.read_csv("/content/hacktest.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Class distribution:")
print(train_df['class'].value_counts())

# Extract NDVI columns
ndvi_columns = [col for col in train_df.columns if '_N' in col]
print(f"Found {len(ndvi_columns)} NDVI features")

X = train_df[ndvi_columns]
y = train_df['class']
X_test = test_df[ndvi_columns]
ids_test = test_df['ID']

# Check for missing values
print(f"Missing values in train: {X.isnull().sum().sum()}")
print(f"Missing values in test: {X_test.isnull().sum().sum()}")

# Advanced imputation with multiple strategies
def advanced_imputation(X_train, X_test):
    """Apply multiple imputation strategies and combine them"""
    # Strategy 1: KNN Imputation
    knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
    X_train_knn = knn_imputer.fit_transform(X_train)
    X_test_knn = knn_imputer.transform(X_test)

    # Strategy 2: Forward/Backward fill (temporal approach)
    X_train_temp = X_train.copy()
    X_test_temp = X_test.copy()

    # Forward fill then backward fill
    X_train_temp = X_train_temp.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
    X_test_temp = X_test_temp.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

    # Combine strategies (weighted average)
    X_train_combined = 0.8 * X_train_knn + 0.2 * X_train_temp.values
    X_test_combined = 0.8 * X_test_knn + 0.2 * X_test_temp.values

    return X_train_combined, X_test_combined

# Apply advanced imputation
X_imputed, X_test_imputed = advanced_imputation(X, X_test)

# Apply noise reduction using Savitzky-Golay filter
def denoise_time_series(X_data):
    """Apply Savitzky-Golay filter to reduce noise"""
    X_denoised = np.zeros_like(X_data)
    for i in range(X_data.shape[0]):
        try:
            # Apply smoothing filter
            X_denoised[i] = savgol_filter(X_data[i], window_length=5, polyorder=2)
        except:
            # If smoothing fails, use original data
            X_denoised[i] = X_data[i]
    return X_denoised

print("Applying noise reduction...")
X_denoised = denoise_time_series(X_imputed)
X_test_denoised = denoise_time_series(X_test_imputed)

# Enhanced feature extraction
def extract_enhanced_features(X_raw):
    """Extract comprehensive features from NDVI time series"""
    df = pd.DataFrame(X_raw, columns=ndvi_columns)
    features = pd.DataFrame()

    # Basic statistical features
    features['mean'] = df.mean(axis=1)
    features['std'] = df.std(axis=1)
    features['min'] = df.min(axis=1)
    features['max'] = df.max(axis=1)
    features['range'] = features['max'] - features['min']
    features['median'] = df.median(axis=1)

    # Percentile features
    features['q25'] = df.quantile(0.25, axis=1)
    features['q75'] = df.quantile(0.75, axis=1)
    features['iqr'] = features['q75'] - features['q25']
    features['skewness'] = df.skew(axis=1)
    features['kurtosis'] = df.kurtosis(axis=1)

    # Temporal features
    features['trend'] = calculate_trend(df)
    features['seasonality'] = features['std'] / (features['mean'] + 0.001)

    # Vegetation-specific features
    features['vegetation_vigor'] = (features['mean'] > 0.3).astype(int)
    features['water_indicator'] = (features['mean'] < 0.1).astype(int)
    features['high_ndvi_count'] = (df > 0.5).sum(axis=1)
    features['low_ndvi_count'] = (df < 0.2).sum(axis=1)

    # Peak and valley detection
    features['peak_count'] = count_peaks(df.values)
    features['valley_count'] = count_valleys(df.values)

    # Growing season features (assuming first half vs second half)
    mid_point = len(ndvi_columns) // 2
    growing_cols = ndvi_columns[:mid_point]
    dormant_cols = ndvi_columns[mid_point:]

    features['growing_mean'] = df[growing_cols].mean(axis=1)
    features['dormant_mean'] = df[dormant_cols].mean(axis=1)
    features['seasonal_diff'] = features['growing_mean'] - features['dormant_mean']
    features['seasonal_ratio'] = features['growing_mean'] / (features['dormant_mean'] + 0.001)

    # Stability features
    features['coefficient_variation'] = features['std'] / (features['mean'] + 0.001)
    features['amplitude'] = features['max'] - features['min']

    return features

def calculate_trend(df):
    """Calculate linear trend for each time series"""
    trends = []
    x = np.arange(len(df.columns))
    for idx in df.index:
        y = df.loc[idx].values
        slope = np.polyfit(x, y, 1)[0]
        trends.append(slope)
    return pd.Series(trends, index=df.index)

def count_peaks(data):
    """Count peaks in time series"""
    peaks = []
    for row in data:
        count = 0
        for i in range(1, len(row)-1):
            if row[i] > row[i-1] and row[i] > row[i+1]:
                count += 1
        peaks.append(count)
    return peaks

def count_valleys(data):
    """Count valleys in time series"""
    valleys = []
    for row in data:
        count = 0
        for i in range(1, len(row)-1):
            if row[i] < row[i-1] and row[i] < row[i+1]:
                count += 1
        valleys.append(count)
    return valleys

# Extract enhanced features
print("Extracting enhanced features...")
X_features = extract_enhanced_features(X_denoised)
X_test_features = extract_enhanced_features(X_test_denoised)

print(f"Number of features: {X_features.shape[1]}")

# Encode target classes
class_mapping = {label: i for i, label in enumerate(sorted(y.unique()))}
inv_class_mapping = {v: k for k, v in class_mapping.items()}
y_encoded = y.map(class_mapping).values

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)
X_test_scaled = scaler.transform(X_test_features)

# Optional: Apply PCA for dimensionality reduction if too many features
if X_scaled.shape[1] > 50:
    print("Applying PCA for dimensionality reduction...")
    pca = PCA(n_components=0.95, random_state=42)  # Keep 95% of variance
    X_scaled = pca.fit_transform(X_scaled)
    X_test_scaled = pca.transform(X_test_scaled)
    print(f"Features reduced to: {X_scaled.shape[1]}")

"""
# Improved ensemble with better hyperparameter tuning
def improved_logistic_ensemble(X, y, X_test, n_models=10, random_state=42):
    Improved ensemble with hyperparameter tuning and better sampling
    skf = StratifiedKFold(n_splits=n_models, shuffle=True, random_state=random_state)

    # Different hyperparameter combinations
    param_combinations = [
        {'C': 0.1, 'solver': 'liblinear', 'multi_class': 'ovr'},
        {'C': 1.0, 'solver': 'liblinear', 'multi_class': 'ovr'},
        {'C': 10.0, 'solver': 'liblinear', 'multi_class': 'ovr'},
        {'C': 0.1, 'solver': 'saga', 'multi_class': 'multinomial'},
        {'C': 1.0, 'solver': 'saga', 'multi_class': 'multinomial'},
        {'C': 10.0, 'solver': 'saga', 'multi_class': 'multinomial'},
        {'C': 0.1, 'solver': 'lbfgs', 'multi_class': 'multinomial'},
        {'C': 1.0, 'solver': 'lbfgs', 'multi_class': 'multinomial'},
        {'C': 10.0, 'solver': 'lbfgs', 'multi_class': 'multinomial'},

    ]

    preds_train = []
    preds_test = []
    model_scores = []
    rng = np.random.default_rng(seed=random_state)

    for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Training model {i+1}/{n_models}")

        # Use different hyperparameters for each model
        params = param_combinations[i % len(param_combinations)]

        # Feature subsampling (use more features for better performance)
        n_features_to_use = max(10, min(X.shape[1], int(0.8 * X.shape[1])))
        feature_indices = rng.choice(X.shape[1], size=n_features_to_use, replace=False)

        # Train model
        clf = LogisticRegression(
            random_state=random_state + i,
            max_iter=2000,
            class_weight='balanced',
            **params
        )

        X_train_subset = X[train_idx][:, feature_indices]
        X_val_subset = X[val_idx][:, feature_indices]

        clf.fit(X_train_subset, y[train_idx])

        # Validate model
        val_pred = clf.predict(X_val_subset)
        val_score = accuracy_score(y[val_idx], val_pred)
        model_scores.append(val_score)
        print(f"Model {i+1} validation accuracy: {val_score:.4f}")

        # Predictions on full datasets
       # preds_train.append(clf.predict(X[:, feature_indices]))
       # preds_test.append(clf.predict(X_test[:, feature_indices]))

        preds_train.append(clf.predict_proba(X[:, feature_indices]))
        preds_test.append(clf.predict_proba(X_test[:, feature_indices]))

    # Convert to arrays
    preds_train = np.array(preds_train)
    preds_test = np.array(preds_test)
    model_scores = np.array(model_scores)

    print(f"Average model accuracy: {model_scores.mean():.4f} (+/- {model_scores.std()*2:.4f})")

    # Weighted ensemble based on validation scores
    weights = model_scores / model_scores.sum()

    # Weighted majority vote
   # y_train_ensemble = weighted_majority_vote(preds_train, weights)
   # y_test_ensemble = weighted_majority_vote(preds_test, weights)

    y_train_ensemble = weighted_soft_vote(preds_train, weights)
    y_test_ensemble = weighted_soft_vote(preds_test, weights)

    return y_train_ensemble, y_test_ensemble, model_scores
"""
def generate_param_combinations(n_samples=10, random_state=42):
    solvers = ['liblinear']
    penalties = {
        'liblinear': ['l1', 'l2'],
        'saga': ['l1', 'l2', 'elasticnet'],
        'lbfgs': ['l2']
    }
    multi_classes = ['ovr']
    Cs = [0.01,0.05, 0.1,0.5, 1.0,5, 10.0,50, 100.0]
    l1_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]

    param_list = []

    for solver in solvers:
        for penalty in penalties[solver]:
            for multi_class in multi_classes:
                for C in Cs:
                    if penalty == 'elasticnet' and solver == 'saga':
                        for l1_ratio in l1_ratios:
                            param_list.append({
                                'solver': solver,
                                'penalty': penalty,
                                'multi_class': multi_class,
                                'C': C,
                                'l1_ratio': l1_ratio
                            })
                    else:
                        param_list.append({
                            'solver': solver,
                            'penalty': penalty,
                            'multi_class': multi_class,
                            'C': C
                        })

    rng = random.Random(random_state)
    return rng.sample(param_list, k=min(n_samples, len(param_list)))

def improved_logistic_ensemble(X, y, X_test, n_models=10, random_state=42):
    """Improved ensemble with randomized hyperparameter tuning and better sampling"""
    skf = StratifiedKFold(n_splits=n_models, shuffle=True, random_state=random_state)
    param_combinations = generate_param_combinations(n_models, random_state)

    preds_train = []
    preds_test = []
    model_scores = []
    rng = np.random.default_rng(seed=random_state)

    for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\nTraining model {i+1}/{n_models}")
        params = param_combinations[i]

        # Feature subsampling (80% features)
        n_features_to_use = max(10, min(X.shape[1], int(0.8 * X.shape[1])))
        feature_indices = rng.choice(X.shape[1], size=n_features_to_use, replace=False)

        clf = LogisticRegression(
            random_state=random_state + i,
            max_iter=2000,
            class_weight='balanced',
            **params
        )

        X_train_subset = X[train_idx][:, feature_indices]
        X_val_subset = X[val_idx][:, feature_indices]

        try:
            clf.fit(X_train_subset, y[train_idx])
        except ValueError as e:
            print(f"Model {i+1} skipped due to error: {e}")
            continue

        val_pred = clf.predict(X_val_subset)
        val_score = accuracy_score(y[val_idx], val_pred)
        model_scores.append(val_score)
        print(f"Validation accuracy: {val_score:.4f}")

        preds_train.append(clf.predict(X[:, feature_indices]))
        preds_test.append(clf.predict(X_test[:, feature_indices]))

    if len(preds_train) == 0:
        raise ValueError("No models successfully trained. Please check your data or parameter space.")

    preds_train = np.array(preds_train)
    preds_test = np.array(preds_test)
    model_scores = np.array(model_scores)
    weights = model_scores / model_scores.sum()

    print(f"\nAverage model accuracy: {model_scores.mean():.4f} (+/- {model_scores.std()*2:.4f})")

    y_train_ensemble = weighted_majority_vote(preds_train, weights)
    y_test_ensemble = weighted_majority_vote(preds_test, weights)

    return y_train_ensemble, y_test_ensemble, model_scores

def weighted_majority_vote(predictions, weights):
    """Weighted majority voting"""
    n_samples = predictions.shape[1]
    n_classes = len(np.unique(predictions))

    ensemble_preds = np.zeros(n_samples, dtype=int)

    for i in range(n_samples):
        # Count weighted votes for each class
        class_votes = np.zeros(n_classes)
        for j, pred in enumerate(predictions[:, i]):
            class_votes[pred] += weights[j]

        # Choose class with highest weighted vote
        ensemble_preds[i] = np.argmax(class_votes)

    return ensemble_preds

def weighted_soft_vote(prob_predictions, weights):
    n_models, n_samples, n_classes = prob_predictions.shape

    # Normalize weights to avoid scaling issues
    weights = np.array(weights) / np.sum(weights)

    # Initialize array to hold final predicted labels
    ensemble_preds = np.zeros(n_samples, dtype=int)

    for i in range(n_samples):
        # Weighted sum of probabilities for each class
        class_probs = np.zeros(n_classes)
        for j in range(n_models):
            class_probs += weights[j] * prob_predictions[j, i]

        # Choose class with highest probability
        ensemble_preds[i] = np.argmax(class_probs)

    return ensemble_preds

# Train improved ensemble
print("Training improved ensemble...")
y_train_pred, y_test_pred, scores = improved_logistic_ensemble(
    X_scaled, y_encoded, X_test_scaled, n_models=7, random_state=42
)

# Evaluate ensemble performance on training data
train_accuracy = accuracy_score(y_encoded, y_train_pred)
print(f"Ensemble training accuracy: {train_accuracy:.4f}")

# Convert predictions back to class labels
y_test_labels = pd.Series(y_test_pred).map(inv_class_mapping)

print("Prediction distribution:")
print(pd.Series(y_test_labels).value_counts())

# Create submission
submission = pd.DataFrame({'ID': ids_test, 'class': y_test_labels})
submission.to_csv('improved_submission.csv', index=False)

print("\nSubmission saved as 'improved_submission.csv'")
print("First few predictions:")
print(submission.head(10))

# Additional validation using train-test split
print("\n" + "="*50)
print("Additional Validation")
print("="*50)

# Split training data for validation
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Train on validation split
y_train_pred_val, y_test_pred_val, val_scores = improved_logistic_ensemble(
    X_train_val, y_train_val, X_test_val, n_models=7, random_state=42
)

# Calculate validation accuracy
val_accuracy = accuracy_score(y_test_val, y_test_pred_val)
print(f"Validation accuracy: {val_accuracy:.4f}")

# Show classification report
y_test_val_labels = [inv_class_mapping[pred] for pred in y_test_val]
y_pred_val_labels = [inv_class_mapping[pred] for pred in y_test_pred_val]

print("\nClassification Report:")
print(classification_report(y_test_val_labels, y_pred_val_labels))

Train shape: (8000, 30)
Test shape: (2845, 29)
Class distribution:
class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64
Found 27 NDVI features
Missing values in train: 25040
Missing values in test: 0
Applying noise reduction...
Extracting enhanced features...
Number of features: 25
Training improved ensemble...

Training model 1/7
Validation accuracy: 0.8110

Training model 2/7
Validation accuracy: 0.8381

Training model 3/7
Validation accuracy: 0.8276

Training model 4/7
Validation accuracy: 0.8311

Training model 5/7
Validation accuracy: 0.8154

Training model 6/7
Validation accuracy: 0.8198

Training model 7/7
Validation accuracy: 0.8187

Average model accuracy: 0.8231 (+/- 0.0177)
Ensemble training accuracy: 0.8333
Prediction distribution:
forest        1813
impervious     440
farm           276
grass          155
water          106
orchard         55
Name: count, dtype: int64

Submission s